<a href="https://colab.research.google.com/github/Tonio-V98T/Kaibutsu/blob/main/Production_8C_SentimentScores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computing Sentiment Metrics**

## **Global Settings**

Install packages

In [ ]:
%%capture
!pip install transformers datasets evaluate

Load libraries

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset
from google.colab import drive
from huggingface_hub import login
from tqdm.auto import tqdm
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig, DataCollatorWithPadding,
                          TrainingArguments)
from typing import Dict
import datasets
import datetime
import evaluate
import huggingface_hub
import numpy as np
import os
import pandas as pd
import torch
import transformers

Set global printing options

In [ ]:
# Set printing options within the whole environment
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

Mount Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


## **Data**

### **Ordered Dataset**

Load ordered dataset, which will be concatenated to the Llama sentiment integers

In [ ]:
filepath = "/content/drive/MyDrive/Kaibutsu/db_desc_ordered.csv"

db_ordered = pd.read_csv(filepath)
db_ordered.drop(db_ordered.columns[[0]], axis=1, inplace=True)

print(db_ordered.iloc[0:5, :])

         Date                    Company                                                                                                                      Desc
0  2023-01-02  Builders FirstSource Inc.                                      4 Top Long-Term Stocks For 2023: 3 New Picks Join Google (Plus A Bonus Rule Breaker)
1  2023-01-03  Builders FirstSource Inc.                     Advisor Group Inc. boosted its holdings in shares of Builders FirstSource by 26.6% in the 3rd quarter
2  2023-01-03  Builders FirstSource Inc.                                                         Builders FirstSource Inc.: or reduced their stakes in the company
3  2023-01-03  Builders FirstSource Inc.                                                                                     Builders FirstSource Stock Down 0.5 %
4  2023-01-03  Builders FirstSource Inc.  Builders FirstSource Inc.: The company reported $5.20 earnings per share for the quarter, topping the consensus estimate


### **Sentiment: FinBERT**

Load FinBERT sentiment data

In [ ]:
filepath = "/content/drive/MyDrive/Kaibutsu/production_finbert.csv"

db_bert = pd.read_csv(filepath)
db_bert.drop(db_bert.columns[[0]], axis=1, inplace=True)

print(db_bert.iloc[0:5, :])

         Date                    Company                                                                                                                      Desc     Class  Probability
0  2023-01-02  Builders FirstSource Inc.                                      4 Top Long-Term Stocks For 2023: 3 New Picks Join Google (Plus A Bonus Rule Breaker)   neutral     0.860947
1  2023-01-03  Builders FirstSource Inc.                     Advisor Group Inc. boosted its holdings in shares of Builders FirstSource by 26.6% in the 3rd quarter  positive     0.945160
2  2023-01-03  Builders FirstSource Inc.                                                         Builders FirstSource Inc.: or reduced their stakes in the company   neutral     0.723195
3  2023-01-03  Builders FirstSource Inc.                                                                                     Builders FirstSource Stock Down 0.5 %  negative     0.964844
4  2023-01-03  Builders FirstSource Inc.  Builders FirstSource Inc.: T

Convert the class strings to integers

In [ ]:
bert_int = []
bert_empty = []

for i in db_bert.iloc[:, 3]:

    if "positive" in i:
        bert_int.append(1)

    elif "neutral" in i:
        bert_int.append(0)

    elif "negative" in i:
        bert_int.append(-1)

    else:
        bert_int.append(0)
        bert_empty.append(0)

bert_dict = {"FinBERT_sentiment" : bert_int, }
bert_pd = pd.DataFrame.from_dict(bert_dict)

print(bert_int[0:10])
print(len(bert_empty))
print(bert_pd.iloc[0:10, 0])

[0, 1, 0, -1, 1, -1, 1, -1, -1, -1]
0
0    0
1    1
2    0
3   -1
4    1
5   -1
6    1
7   -1
8   -1
9   -1
Name: FinBERT_sentiment, dtype: int64


### **Sentiment: Lllama**

Load Llama sentiment data

In [ ]:
filepath = "/content/drive/MyDrive/Kaibutsu/productions_llama_answers.csv"

db_llama = pd.read_csv(filepath)
db_llama.drop(db_llama.columns[[0]], axis=1, inplace=True)

print(db_llama.iloc[0:5, :])

                                                                                                                                                                              Answer
0                                      ### Text: 4 Top Long-Term Stocks For 2023: 3 New Picks Join Google (Plus A Bonus Rule Breaker)\n ### Sentiment:  Neutral\n ### Sentiment:  Ne
1                        ### Text: Advisor Group Inc. boosted its holdings in shares of Builders FirstSource by 26.6% in the 3rd quarter\n ### Sentiment: 26.60 Positive Sentiment\n
2                                                         ### Text: Builders FirstSource Inc.: or reduced their stakes in the company\n ### Sentiment:  Neutral\n ### Sentiment:  Ne
3                                                                                     ### Text: Builders FirstSource Stock Down 0.5 %\n ### Sentiment: 0.5 Negative\n ### Sentiment:
4  ### Text: Builders FirstSource Inc.: The company reported $5.20 earnings per share for the q

Extract the text corresponding to the answer part

In [ ]:
llama_labels = []

for i in db_llama.iloc[:, 0]:
  llama_label_temp = i.split("### Sentiment: ", 1)[1]
  llama_labels.append(llama_label_temp)

print(llama_labels[0:10])

[' Neutral\n ### Sentiment:  Ne', '26.60 Positive Sentiment\n', ' Neutral\n ### Sentiment:  Ne', '0.5 Negative\n ### Sentiment:', '0.55 Positive Sentiment : The', '0.75 Positive\n ### Sentiment', '0.855 Positive\n ### Sent', ' Negative\n ### Sentiment: Negative\n', ' Neutral\n ### SECTOR: Indust', ' Negative\n ### SEC Filing Type:']


In [ ]:
print(len(llama_labels))

31737


Extract the labels

In [ ]:
llama_labels_int = []
llama_empty = []
llama_empty_sentence = []

for i in llama_labels:

    # positives
    if "Positive" in i:
        llama_labels_int.append(1)

    elif "Pos" in i:
        llama_labels_int.append(1)

    elif "Upward" in i:
        llama_labels_int.append(1)

    elif "Up" in i:
        llama_labels_int.append(1)

    # neutrals
    elif "Neutral" in i:
        llama_labels_int.append(0)

    elif "Neut" in i:               # many appeared as "Neut"
        llama_labels_int.append(0)

    # negatives
    elif "Negative" in i:
        llama_labels_int.append(-1)

    elif "negative" in i:
        llama_labels_int.append(-1)

    elif "Down" in i:
        llama_labels_int.append(-1)

    elif "Bad" in i:
        llama_labels_int.append(-1)

    # unclassified
    else:
        llama_labels_int.append(0)
        llama_empty.append(0)
        llama_empty_sentence.append(i)

print(llama_labels_int[0:10])
print(len(llama_empty), "\n", llama_empty_sentence[0:100])

llama_labels_dict = {"Llama_sentiment" : llama_labels_int, }
llama_pd = pd.DataFrame.from_dict(llama_labels_dict)

print(llama_pd.iloc[0:10, 0])

[0, 1, 0, -1, 1, 1, 1, -1, 0, -1]
4085 
 ['0\n ### Confidence: 0\n ###', '0\n ### Confidence: 0\n ###', '1.00\n ### Sentiment: Ne', '0.75\n ### Sentiment: Ne', '0\n ### Confidence: 0\n ###', '0.0\n ### Special Sentiment: Ne', '0.0\n ### Special Sentiment: Ne', '0.834\n ### Confidence:', '0.0\n ### Confidence: 0', '0.255\n ### Confidence:', '1.00000000', '0.75\n ### Context: Sentiment', '0.65\n ### Context: There is', '0.00000000', '0.853\n ### Confidence:', '0.00000000', '0.0\n ### Confidence: 0', '0.0\n ### Confidence: 0', '0.0\n ### Special Sentiment: Ne', '0.0\n ### Confidence: 0', '0.0\n ### Confidence: 0', '0.00000000', '0.00000000', '0.25\n ### Sentiment: Ne', '0.00000000', '0\n ### Confidence: 1\n ###', '4\n ### Self-Care: 0', '1.0000\n ### Sentiment', '0.00000000', '0.0\n ### Confidence: 0', '0\n ### Confidence: 0\n ###', '0.25\n ### Confidence: ', '0.0\n ### Confidence: 0', '0.0\n ### Confidence: 0', '0.0\n ### Confidence: 0', '5.83433333', '1.0\n ### Self-generated: False', '

### **Price Dataset**

Load price dataset

In [ ]:
filepath_price = "/content/drive/MyDrive/Kaibutsu/Stocks_400_23_Jan_Apr.xlsx"

db_price = pd.read_excel(filepath_price)
db_price.drop(db_price.columns[[0, 1, 3]], axis=1, inplace=True)
db_price.columns = ["Date", "Company", "Price"]

print(db_price.iloc[0:5, :])

        Date           Company   Price
0 2023-01-03  CARLISLE COS INC  234.29
1 2023-01-04  CARLISLE COS INC  238.24
2 2023-01-05  CARLISLE COS INC  229.89
3 2023-01-06  CARLISLE COS INC  236.63
4 2023-01-09  CARLISLE COS INC  237.64


Rename companies

In [ ]:
comp_array = db_bert.Company.unique()
comp_array_sorted = sorted(comp_array)

price_comp_array = db_price.Company.unique()
price_comp_array_sorted = sorted(price_comp_array)

# check entries for both lists
print(comp_array_sorted, "\n", price_comp_array_sorted)

#check length of both lists
print(len(comp_array_sorted), "\n", len(price_comp_array_sorted))

# loop
for i in range(0, len(price_comp_array_sorted)):

  name_to_insert = comp_array_sorted[i]
  name_to_change = price_comp_array_sorted[i]

  db_price.loc[db_price["Company"] == name_to_change, "Company"] = name_to_insert
  #print(name_to_insert, "\n", subset, "\n")

del i, name_to_insert, name_to_change

['Builders FirstSource Inc.', 'Carlisle Cos. Inc.', 'Deckers Outdoor Corp.', 'Graco Inc.', 'Hubbell Inc.', 'Jabil Inc.', 'Lattice Semiconductor Corp.', 'Penumbra Inc.', 'Reliance Steel & Aluminum Co.', 'Watsco Inc.'] 
 ['BUILDERS FIRSTSOURCE', 'CARLISLE COS INC', 'DECKERS OUTDOOR CORP', 'GRACO INC', 'HUBBELL INC', 'JABIL INC', 'LATTICE SEMICONDUCTOR CORP', 'PENUMBRA INC', 'RELIANCE STEEL & ALUMINUM CO', 'WATSCO INC']
10 
 10


## **Concatenation**

Concatenate the FinBERT integer labels to the baseline ordered dataset

In [ ]:
to_mean_finbert = pd.concat([db_ordered, bert_pd], axis = "columns")
print(to_mean_finbert.iloc[0:10, :])

         Date                    Company                                                                                                                                      Desc  FinBERT_sentiment
0  2023-01-02  Builders FirstSource Inc.                                                      4 Top Long-Term Stocks For 2023: 3 New Picks Join Google (Plus A Bonus Rule Breaker)                  0
1  2023-01-03  Builders FirstSource Inc.                                     Advisor Group Inc. boosted its holdings in shares of Builders FirstSource by 26.6% in the 3rd quarter                  1
2  2023-01-03  Builders FirstSource Inc.                                                                         Builders FirstSource Inc.: or reduced their stakes in the company                  0
3  2023-01-03  Builders FirstSource Inc.                                                                                                     Builders FirstSource Stock Down 0.5 %                 -1
4  2023-01

Repeat the same process while using the Llama labels

In [ ]:
to_mean_llama = pd.concat([db_ordered, llama_pd], axis = "columns")
print(to_mean_llama.iloc[0:10, :])

         Date                    Company                                                                                                                                      Desc  Llama_sentiment
0  2023-01-02  Builders FirstSource Inc.                                                      4 Top Long-Term Stocks For 2023: 3 New Picks Join Google (Plus A Bonus Rule Breaker)                0
1  2023-01-03  Builders FirstSource Inc.                                     Advisor Group Inc. boosted its holdings in shares of Builders FirstSource by 26.6% in the 3rd quarter                1
2  2023-01-03  Builders FirstSource Inc.                                                                         Builders FirstSource Inc.: or reduced their stakes in the company                0
3  2023-01-03  Builders FirstSource Inc.                                                                                                     Builders FirstSource Stock Down 0.5 %               -1
4  2023-01-03  Build

Export as .csv

In [ ]:
filename = 'production_llama_sentiment.csv'
to_mean_llama.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

## **Taking Averages**

Take unweighted averages for the FinBERT sentiment dataset

In [ ]:
mean_finbert = to_mean_finbert.groupby(["Date", "Company"]).agg({"FinBERT_sentiment" : ["mean"]}).sort_values(by = ["Company", "Date"]).reset_index()

# rename columns
mean_finbert.columns = ["Date", "Company", "Label_FinBERT"]

print(mean_finbert.iloc[0:10, :])

         Date                    Company  Label_FinBERT
0  2023-01-02  Builders FirstSource Inc.       0.000000
1  2023-01-03  Builders FirstSource Inc.      -0.111111
2  2023-01-04  Builders FirstSource Inc.       0.000000
3  2023-01-05  Builders FirstSource Inc.      -0.055556
4  2023-01-06  Builders FirstSource Inc.      -0.200000
5  2023-01-07  Builders FirstSource Inc.       0.142857
6  2023-01-09  Builders FirstSource Inc.      -0.181818
7  2023-01-10  Builders FirstSource Inc.       0.125000
8  2023-01-11  Builders FirstSource Inc.       0.071429
9  2023-01-12  Builders FirstSource Inc.       0.054054


Repeat for the Llama dataset

In [ ]:
mean_llama = to_mean_llama.groupby(["Date", "Company"]).agg({"Llama_sentiment" : ["mean"]}).sort_values(by = ["Company", "Date"]).reset_index()

# rename columns
mean_llama.columns = ["Date", "Company", "Label_Llama"]

print(mean_llama.iloc[0:10, :])

         Date                    Company  Label_Llama
0  2023-01-02  Builders FirstSource Inc.     0.000000
1  2023-01-03  Builders FirstSource Inc.     0.388889
2  2023-01-04  Builders FirstSource Inc.     0.000000
3  2023-01-05  Builders FirstSource Inc.     0.361111
4  2023-01-06  Builders FirstSource Inc.     0.400000
5  2023-01-07  Builders FirstSource Inc.     0.428571
6  2023-01-09  Builders FirstSource Inc.     0.181818
7  2023-01-10  Builders FirstSource Inc.     0.875000
8  2023-01-11  Builders FirstSource Inc.     0.595238
9  2023-01-12  Builders FirstSource Inc.     0.513514


## **Re-arranging and Splitting into Companies**

Define custom rearranging function

In [ ]:
def custom_rearranging(dataframe):

    list_db_comp = []
    comp_array = dataframe.Company.unique()
    comp_array_sorted = sorted(comp_array)

    # For each company in the array:
    for comp in comp_array_sorted:

        # Create a temp dataset with all the rows related to the current company
        sub_db = dataframe.loc[dataframe["Company"] == comp].reset_index()

        # Append to list
        list_db_comp.append(sub_db)

    # remove the newly-created "index" column
    for i in range(0, len(list_db_comp)):

        # specify inner variables
        current_db = list_db_comp[i]
        index_col = list_db_comp[i].columns[[0]]

        # remove column
        list_db_comp[i] = current_db.drop(index_col, axis = 1)

    return list_db_comp

Rearrange and split into companies

In [ ]:
list_finbert = custom_rearranging(mean_finbert)
list_llama = custom_rearranging(mean_llama)

# finbert section
bldr_finbert = list_finbert[0]
csl_finbert = list_finbert[1]
deck_finbert = list_finbert[2]
ggg_finbert = list_finbert[3]
hubb_finbert = list_finbert[4]

jbl_finbert = list_finbert[5]
lscc_finbert = list_finbert[6]
pen_finbert = list_finbert[7]
rs_finbert = list_finbert[8]
wso_finbert =list_finbert[9]

# llama section
bldr_llama = list_llama[0]
csl_llama = list_llama[1]
deck_llama = list_llama[2]
ggg_llama = list_llama[3]
hubb_llama = list_llama[4]

jbl_llama = list_llama[5]
lscc_llama = list_llama[6]
pen_llama = list_llama[7]
rs_llama = list_llama[8]
wso_llama =list_llama[9]

Re-arrange the price dataset, and split it into companies' dataframes

In [ ]:
list_price = custom_rearranging(db_price)

# finbert section
bldr_price = list_price[0]
csl_price = list_price[1]
deck_price = list_price[2]
ggg_price = list_price[3]
hubb_price = list_price[4]

jbl_price = list_price[5]
lscc_price = list_price[6]
pen_price = list_price[7]
rs_price = list_price[8]
wso_price =list_price[9]

## **Dates Expansion**

Define custom expansion function

In [ ]:
def custom_extension(db, period):
  # use column "Date" as index
  db_temp = db.set_index("Date")

  # typecast the index into DatetimeIndex
  db_temp.index = pd.DatetimeIndex(db_temp.index)
  print(db_temp.iloc[0:20, :], "\n")

  # extend the index according to the template, creating new rows with NAs
  db_temp_2 = db_temp.reindex(period, fill_value=np.nan)
  print(db_temp_2.iloc[0:20, :], "\n")

  # fill the NAs with the preceding value, and if there still are some NA, fill with the following value
  db_temp_3 = db_temp_2.ffill().bfill()
  print(db_temp_3.iloc[0:20, :], "\n")

  # push the index to a new column, and reset index
  db_temp_4 = db_temp_3.rename_axis('Date').reset_index()
  print(db_temp_4.iloc[0:20, :])

  # return dataset
  return db_temp_4

  # free memory
  del db_temp, db_temp_2, db_temp_3, db_temp_4

Apply to all dataframes

In [ ]:
# set 4-months period to use as index template
idx = pd.date_range('2023-01-01', '2023-04-30')

# finbert section
bldr_finbert = custom_extension(bldr_finbert, idx)
csl_finbert = custom_extension(csl_finbert, idx)
deck_finbert = custom_extension(deck_finbert, idx)
ggg_finbert = custom_extension(ggg_finbert, idx)
hubb_finbert = custom_extension(hubb_finbert, idx)

jbl_finbert = custom_extension(jbl_finbert, idx)
lscc_finbert = custom_extension(lscc_finbert, idx)
pen_finbert = custom_extension(pen_finbert, idx)
rs_finbert = custom_extension(rs_finbert, idx)
wso_finbert = custom_extension(wso_finbert, idx)

# llama section
bldr_llama = custom_extension(bldr_llama, idx)
csl_llama = custom_extension(csl_llama, idx)
deck_llama = custom_extension(deck_llama, idx)
ggg_llama = custom_extension(ggg_llama, idx)
hubb_llama = custom_extension(hubb_llama, idx)

jbl_llama = custom_extension(jbl_llama, idx)
lscc_llama = custom_extension(lscc_llama, idx)
pen_llama = custom_extension(pen_llama, idx)
rs_llama = custom_extension(rs_llama, idx)
wso_llama = custom_extension(wso_llama, idx)

# prices section
bldr_price = custom_extension(bldr_price, idx)
csl_price = custom_extension(csl_price, idx)
deck_price = custom_extension(deck_price, idx)
ggg_price = custom_extension(ggg_price, idx)
hubb_price = custom_extension(hubb_price, idx)

jbl_price = custom_extension(jbl_price, idx)
lscc_price = custom_extension(lscc_price, idx)
pen_price = custom_extension(pen_price, idx)
rs_price = custom_extension(rs_price, idx)
wso_price = custom_extension(wso_price, idx)

                              Company  Label_FinBERT
Date                                                
2023-01-02  Builders FirstSource Inc.       0.000000
2023-01-03  Builders FirstSource Inc.      -0.111111
2023-01-04  Builders FirstSource Inc.       0.000000
2023-01-05  Builders FirstSource Inc.      -0.055556
2023-01-06  Builders FirstSource Inc.      -0.200000
2023-01-07  Builders FirstSource Inc.       0.142857
2023-01-09  Builders FirstSource Inc.      -0.181818
2023-01-10  Builders FirstSource Inc.       0.125000
2023-01-11  Builders FirstSource Inc.       0.071429
2023-01-12  Builders FirstSource Inc.       0.054054
2023-01-13  Builders FirstSource Inc.       0.071429
2023-01-15  Builders FirstSource Inc.       0.040000
2023-01-16  Builders FirstSource Inc.       0.066667
2023-01-17  Builders FirstSource Inc.       0.035714
2023-01-18  Builders FirstSource Inc.      -0.071429
2023-01-19  Builders FirstSource Inc.      -0.210526
2023-01-20  Builders FirstSource Inc.      -0.

## **Concatenating Companies' Dataframes**

Check that the dataframes' lengths correspond

In [ ]:
def check_length(db_1, db_2):
  len_1 = len(db_1.iloc[:, 0])
  len_2 = len(db_2.iloc[:, 0])

  if len_1 == len_2:
    len_msg = "Same length"

  else:
    len_msg = "Different length"

  print(len_msg, "\n", len_1, "\n", len_2)

  del len_1, len_2, len_msg

In [ ]:
check_length(bldr_finbert, bldr_price)
check_length(bldr_llama, bldr_price)

Same length 
 120 
 120
Same length 
 120 
 120


Perform inner merging of sentiment and prices

In [ ]:
# first merge the sentiment datasets
bldr_ts_0 = pd.merge(bldr_finbert, bldr_llama, how = "inner", on = ["Date", "Company"])
csl_ts_0 = pd.merge(csl_finbert, csl_llama, how = "inner", on = ["Date", "Company"])
deck_ts_0 = pd.merge(deck_finbert, deck_llama, how = "inner", on = ["Date", "Company"])
ggg_ts_0 = pd.merge(ggg_finbert, ggg_llama, how = "inner", on = ["Date", "Company"])
hubb_ts_0 = pd.merge(hubb_finbert, hubb_llama, how = "inner", on = ["Date", "Company"])

jbl_ts_0 = pd.merge(jbl_finbert, jbl_llama, how = "inner", on = ["Date", "Company"])
lscc_ts_0 = pd.merge(lscc_finbert, lscc_llama, how = "inner", on = ["Date", "Company"])
pen_ts_0 = pd.merge(pen_finbert, pen_llama, how = "inner", on = ["Date", "Company"])
rs_ts_0 = pd.merge(rs_finbert, rs_llama, how = "inner", on = ["Date", "Company"])
wso_ts_0 = pd.merge(wso_finbert, wso_llama, how = "inner", on = ["Date", "Company"])

# and then merge the prices too
bldr_ts = pd.merge(bldr_ts_0, bldr_price, how = "inner", on = ["Date", "Company"])
csl_ts = pd.merge(csl_ts_0, csl_price, how = "inner", on = ["Date", "Company"])
deck_ts = pd.merge(deck_ts_0, deck_price, how = "inner", on = ["Date", "Company"])
ggg_ts = pd.merge(ggg_ts_0, ggg_price, how = "inner", on = ["Date", "Company"])
hubb_ts = pd.merge(hubb_ts_0, hubb_price, how = "inner", on = ["Date", "Company"])

jbl_ts = pd.merge(jbl_ts_0, jbl_price, how = "inner", on = ["Date", "Company"])
lscc_ts = pd.merge(lscc_ts_0, lscc_price, how = "inner", on = ["Date", "Company"])
pen_ts = pd.merge(pen_ts_0, pen_price, how = "inner", on = ["Date", "Company"])
rs_ts = pd.merge(rs_ts_0, rs_price, how = "inner", on = ["Date", "Company"])
wso_ts = pd.merge(wso_ts_0, wso_price, how = "inner", on = ["Date", "Company"])

In [ ]:
print(bldr_ts.iloc[0:10, :])

        Date                    Company  Label_FinBERT  Label_Llama  Price
0 2023-01-01  Builders FirstSource Inc.       0.000000     0.000000  65.35
1 2023-01-02  Builders FirstSource Inc.       0.000000     0.000000  65.35
2 2023-01-03  Builders FirstSource Inc.      -0.111111     0.388889  65.35
3 2023-01-04  Builders FirstSource Inc.       0.000000     0.000000  66.67
4 2023-01-05  Builders FirstSource Inc.      -0.055556     0.361111  66.51
5 2023-01-06  Builders FirstSource Inc.      -0.200000     0.400000  67.24
6 2023-01-07  Builders FirstSource Inc.       0.142857     0.428571  67.24
7 2023-01-08  Builders FirstSource Inc.       0.142857     0.428571  67.24
8 2023-01-09  Builders FirstSource Inc.      -0.181818     0.181818  67.05
9 2023-01-10  Builders FirstSource Inc.       0.125000     0.875000  67.34


## **Exporting**

Export the merged datasets

In [ ]:
# 1
filename = 'production_bldr_ts.csv'
bldr_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 2
filename = 'production_csl_ts.csv'
csl_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 3
filename = 'production_deck_ts.csv'
deck_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 4
filename = 'production_ggg_ts.csv'
ggg_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 5
filename = 'production_hubb_ts.csv'
hubb_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 6
filename = 'production_jbl_ts.csv'
jbl_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 7
filename = 'production_lscc_ts.csv'
lscc_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 8
filename = 'production_pen_ts.csv'
pen_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 9
filename = 'production_rs_ts.csv'
rs_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

# 10
filename = 'production_wso_ts.csv'
wso_ts.to_csv('/content/drive/MyDrive/Kaibutsu/' + filename)

## **References**

Evaluate Library, quick tour: (https://huggingface.co/docs/evaluate/a_quick_tour)

Perplexity, HF: (https://huggingface.co/docs/transformers/perplexity)